In [24]:
from google import genai
from google.genai.types import GenerateContentConfig
import os, json

api_key = os.getenv("GEMINI_API_KEY")

client = genai.Client(api_key=api_key)

tools = [
    {"url_context": {}},
    {"google_search": {}}
]



url = "https://www.youtube.com/watch?v=THTmDBtAlf4"
language = "Vietnamese"



prompt = f"""
ROLE
You are an assistant that summarizes YouTube videos.

TASK
Given a YouTube video URL, analyze the video content and generate a structured summary. You may infer information from the video, including title, description, chapters, subtitles, and spoken content if available.

Then do the following:
- Return ONLY valid JSON
- Do NOT include explanations or extra text
- You can process this task by every language but OUPUTPUT LANGUAGE MUST{language}
- Use Markdown-compatible bullet points ("-")
- Ensure the structure strictly follows the schema below
- Limit total output to a maximum of 30,000 tokens
- If the video is very long, limit the number of segments so the output does not exceed this limit
- Only provide a concise and representative selection of the most important insights and summary segments as needed

Youtube Video URL:
{url}

JSON SCHEMA
{{
  "insights": [
    {{
      "insight": "Insight 1",
      "sub-insights": [
        "Insight point 1",
        "Insight point 2"
      ]
    }}
  ],
  "summary": {{
    "tldr": "One line summary of the video",
    "timestamps": [
      {{
        "time": "MM:SS",
        "start_seconds": 0,
        "text": "Summary segment 1"
      }}
    ]
  }}
}}

FINAL INSTRUCTION
Generate the JSON now.
"""

import json
import re

response = client.models.generate_content(
    model="models/gemini-2.5-flash-lite",
    contents=prompt,
    config=GenerateContentConfig(
        tools=tools
    )
)

# --- Extract text from parts ---
raw = ""
if response.candidates:
    for part in response.candidates[0].content.parts:
        if hasattr(part, "text") and part.text:
            raw += part.text

raw = raw.strip()

# --- Clean markdown json block nếu có ---
raw = re.sub(r"^```json|```$", "", raw, flags=re.MULTILINE).strip()

if not raw:
    print("❌ No text output from model. Possibly tool call only.")
    print(response)
else:
    data = json.loads(raw)  # Nếu lỗi JSON -> chương trình sẽ throw exception luôn
    print(json.dumps(data, indent=2, ensure_ascii=False))



{
  "insights": [
    {
      "insight": "Khói - Tháng 7 của anh, em và cô đơn",
      "sub-insights": [
        "Bài hát này nói về nỗi buồn và sự cô đơn sau khi chia tay.",
        "Người hát cố gắng che giấu sự yếu đuối và những ký ức đẹp với người yêu cũ.",
        "Dù đau khổ, người hát vẫn chúc phúc cho hạnh phúc của người mình yêu."
      ]
    }
  ],
  "summary": {
    "tldr": "Bài hát \"Tháng 7 của anh, em và cô đơn\" của Khói là một bản rap tâm trạng về nỗi buồn, sự cô đơn và những kỷ niệm sau chia tay, với thông điệp chúc phúc cho người mình yêu dù không còn ở bên nhau.",
    "timestamps": [
      {
        "time": "01:27",
        "start_seconds": 87,
        "text": "Người hát hồi tưởng về những kỷ niệm đẹp và sự chân thành trong tình yêu đã qua."
      },
      {
        "time": "03:01",
        "start_seconds": 181,
        "text": "Anh thừa nhận sự mâu thuẫn trong cảm xúc và nỗi cô đơn đeo bám."
      },
      {
        "time": "04:10",
        "start_seconds": 250,
   